In [ ]:
#using unsupervied learning model to predit the average rating for #1 restaurant category in USA
#Neural Networks with 1 layer and 1 neuron

import json
import pandas as pd
import tensorflow as tf
import warnings
warnings.simplefilter("ignore")

from matplotlib import pyplot as plt
from pprint import pprint
from sklearn.cluster import DBSCAN
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split

In [ ]:
#read in dataset
df = pd.read_csv('cleaned_data.csv')
df.head()

In [ ]:
#separate American Restaurents with others
df['category'] = df['category'].fillna('')

american_restaurants = df[df['category'].str.contains('American', case=False)]
other_restaurants = df[~df['category'].str.contains('American', case=False)]


In [ ]:
#Drop irrelative columns and rows with no value
columns_to_drop = ['business_id', 'category', 'review_id','user_id']
X = other_restaurants.drop(columns=columns_to_drop)
X = X.dropna()


In [ ]:
#Run DBSCAN model
%%time
# CAUTION! This step may take several hours
model = DBSCAN(eps=10, min_samples=20,metric='l1')
model.fit(X)

In [ ]:
model.labels_

In [ ]:
#Get number to components
len(model.components_)


In [ ]:
#View labels
pd.Series(model.labels_).value_counts()

In [ ]:
#Add labels to the dataframe
labeled_df = X.copy()
labeled_df['label'] = model.labels_ + 1
labeled_df

In [ ]:
#merge labeled df and original df
merged_df = pd.merge(X, labeled_df, on=['latitude', 'longitude'], how='inner')
merged_df

In [ ]:
#create pivot table for average rating to each clusters
mean_ratings = df.groupby('user_id')['stars'].mean()
mean_ratings
#user_rating_mapping = mean_ratings.to_dict()

#mean_ratings_by_label = labeled_df.groupby(['label'])['stars'].mean().reset_index()

#new_df = mean_ratings_by_label.pivot_table(values='stars', fill_value=0)

#new_df.columns = [f"Label_{label}" for label in new_df.columns]

#new_df
#new_df = df2[df2['user_id'].isin(top_italian_restaurant_reviewers.index)].groupby(['user_id', 'label']).mean().reset_index().set_index('user_id').pivot(columns='label', values='stars')

In [ ]:
X = new_df.drop(-1, axis=1).fillna(3)
y = new_df[-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
nn_model = tf.keras.Sequential()
nn_model.add(tf.keras.layers.Dense(units=1, input_dim=len(X.columns)))
nn_model.compile(loss="MSE", optimizer="adam", metrics=["mse", "mae"])
fit_model = nn_model.fit(X_train, y_train, epochs=50)

In [ ]:
nn_model.save("prediction_model.h5")


In [ ]:
pd.DataFrame(fit_model.history, index=range(1,len(fit_model.history["loss"])+1)).plot(y="mean_squared_error")


In [ ]:
r2_score(y_test, nn_model.predict(X_test))


In [ ]:
mean_absolute_error(y_test, nn_model.predict(X_test))


In [ ]:
plt.scatter(nn_model.predict(X_test), y_test)
